In [1]:
import requests
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import time
from tqdm import tqdm
import pickle

Query https://chessdb.cn/queryc_en/ to get the engine moves from a given postion

In [2]:
base_url = 'http://www.chessdb.cn/cdb.php'

def process_fen(fen):
    params = {
        'action': 'queryall',
        'board': fen,
        'showall': 1
    }

    # Send the GET request
    response = requests.get(base_url, params=params)

    moves_data = response.text.strip().split('|')
    engine_moves = []
    max_score = -np.inf
    for move_data in moves_data:
        #Convert the query into the relevant data
        details = move_data.split(',')
        move = details[0].split(':')[1]
        score = details[1].split(':')[1]

        #Keep all moves with the top score
        if (score.isnumeric() or (score.startswith('-') and score[1:].isnumeric())) and int(score) >= max_score:
            max_score = int(score)
            engine_moves.append(move)
        else:
            break
        
    return engine_moves

def process_multiple_fens(fens, max_workers=10):
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(tqdm(executor.map(process_fen, fens), total=len(fens)))
    return results

Load in a 1,000,000 positions to query

In [4]:
index = 0
with open(f'chess_data/move_dict_{index:03d}.pkl', 'rb') as file:
    # Load the dictionary from the file
    move_dict = pickle.load(file)

In [ ]:
opening_fens = []

#Remove the FENs that are in common openings since humans and engines
#will have similar suggestions here so this data is not as valuable
fens = [fen for fen in move_dict.keys() if fen not in opening_fens]

engine_moves_list = process_multiple_fens(fens)


Save the engine move dictionary

In [ ]:
engine_move_dicts = {key:value for key, value in zip(fens, engine_moves_list)}

with open(f'chess_data/engine_move_dict_{index:03d}.pkl', 'wb') as f:
    pickle.dump(engine_move_dicts, f)